In [1]:
import twint
import nest_asyncio
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

nest_asyncio.apply()

### Before

In [ ]:
c = twint.Config()
c.Username = 'infoJakarta'
c.Since = "2020-08-01" #batas awal tanggal pengambilan tweet
c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
for key in ['vaksin', 'covid']:
    c.Search = key
    c.Output = './data/before.json' # save to json
    twint.run.Search(c)

### After

In [ ]:
c = twint.Config()
c.Username = 'infoJakarta'
c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
c.Count = True         
c.Retweets = True
c.Stats = True 
c.Show_hashtags = True
c.Hide_output = True
c.Profile_full = True
c.Store_json = True
for key in ['vaksin', 'covid']:
    c.Search = key
    c.Output = './data/after.json' # save to json
    twint.run.Search(c)

In [ ]:
df_before = pd.read_json('./data/before.json' , lines = True)
df_after = pd.read_json('./data/after.json' , lines = True)

In [ ]:
# df_before.duplicated()
df_before = df_before[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

df_before = df_before.drop_duplicates(subset='link')
for i in df_before['link']:
    print(i)

In [2]:
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
# from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
import numpy as np
from datetime import datetime, timedelta

In [12]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
# chrome_options.add_argument('headless') # untuk menutup browser

#specify the path to chromedriver.exe
driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

<ipython-input-12-4a1c4cd5d846>:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)


In [13]:
driver.get('https://www.twitter.com/login')
sleep(2)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('dyingrabbit_') # Ubah username sesuai akun pribadi

my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

········


In [ ]:
data_tweets_before = df_before

In [14]:
def get_list_replies(link):
    list_replies = []
    driver.get(link)
    sleep(2)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)
    
    try:
        replies_container = driver.find_element_by_xpath('//div[@aria-label="Timeline: Conversation"]')
        if replies_container:
            replies = replies_container.find_elements_by_xpath('.//div[@dir="ltr"]/span')
            for reply in replies:
                list_replies.append(reply.text)
    except:
        print(list_replies)
        return list_replies
    print(list_replies)
    return list_replies

In [ ]:
arr_temp = []
for i, row in data_tweets_before.iterrows():
     arr_temp.append(get_list_replies(row['link']))

data_tweets_before['list_replies'] = arr_temp
data_tweets_before['list_replies']

In [ ]:
data_tweets_before.to_json('./data/before_replies.json')

In [ ]:
data_tweets_before = pd.read_json('./data/before_replies.json')
data_tweets_before

In [ ]:
arr = []
for i in data_tweets_before['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
arr.remove('@infojakarta')
arr

In [ ]:
filename = './data/before_user_data.json'
for uname in arr:
    uname = uname[1:]
    c = twint.Config()
    c.Username = uname
    c.Since = "2020-08-01" #batas awal tanggal pengambilan tweet
    c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
    c.Search = "covid OR vaksin"
    c.Count = True         
    c.Retweets = True
    c.Stats = True 
    c.Show_hashtags = True
    c.Hide_output = True
    c.Profile_full = True
    c.Store_json = True
    c.Output = filename # save to json
    twint.run.Search(c)

In [ ]:
before_user_data_tweets = pd.read_json('./data/before_user_data.json', lines = True)
before_user_data_tweets = before_user_data_tweets[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

before_user_data_tweets = before_user_data_tweets.drop_duplicates(subset='link')
for i in before_user_data_tweets['link']:
    print(i)

In [ ]:
arr_temp = []
for i, row in before_user_data_tweets.iterrows():
     arr_temp.append(get_list_replies(row['link']))

before_user_data_tweets['list_replies'] = arr_temp
before_user_data_tweets['list_replies']

In [ ]:
before_user_data_tweets.to_json('./data/before_user_scrap.json')

### SCRAPPING AFTER DATA

In [ ]:
df_after = pd.read_json('./data/after.json' , lines = True)
df_after = df_after[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

df_after = df_after.drop_duplicates(subset='link')
for i in df_after['link']:
    print(i)

In [ ]:
data_tweets_after = df_after

In [ ]:
arr_temp = []
for i, row in data_tweets_after.iterrows():
     arr_temp.append(get_list_replies(row['link']))

data_tweets_after['list_replies'] = arr_temp
data_tweets_after['list_replies']

In [ ]:
data_tweets_after.to_json('./data/after_replies.json')

In [7]:
data_tweets_after = pd.read_json('./data/after_replies.json')
data_tweets_after

,id,conversation_id,user_id,username,name,tweet,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,user_rt_id,user_rt,retweet_id,reply_to,list_replies
0,1384215782289141767,1384215782289141760,106038219,infojakarta,@InfoJakarta,Gratis vaksin non lansia ...? Kan memang vaks...,3,5,2,[],[],https://twitter.com/infojakarta/status/1384215...,False,https://twitter.com/dinkesJKT/status/138209661...,,,,[],"[@infojakarta, @dinkesJKT, Apr 14, @eckaNh57]"
1,1376929493814677504,1376929493814677504,106038219,infojakarta,@InfoJakarta,"Kurang lebihnya. Sembilan orang ditest, satu ...",3,0,1,[],[],https://twitter.com/infojakarta/status/1376929...,False,https://twitter.com/DKIJakarta/status/13768936...,,,,[],"[@infojakarta, @DKIJakarta, Mar 30, @melwth_, ..."
2,1376883129890205696,1376883129890205696,106038219,infojakarta,@InfoJakarta,Kapan kira-kira kamu dapat vaksin?,33,1,13,[],[],https://twitter.com/infojakarta/status/1376883...,False,,,,,[],"[@infojakarta, @fyrdausy, @RemajaKolonial, @Li..."
3,1373529155304706054,1373529155304706048,106038219,infojakarta,@InfoJakarta,Kalau tidak salah baca datanya. Kemarin 20 ma...,2,4,5,[],[],https://twitter.com/infojakarta/status/1373529...,False,https://twitter.com/DKIJakarta/status/13732387...,,,,[],"[@infojakarta, @DKIJakarta, Mar 20, @chiko_jub..."
4,1372218371442319360,1372218371442319360,106038219,infojakarta,@InfoJakarta,Vaksin lansia di @UnikaAtmaJaya Terbuka untuk...,0,12,13,[],[],https://twitter.com/infojakarta/status/1372218...,False,,,,,[],[@infojakarta]
5,1372150088248664068,1372150088248664064,106038219,infojakarta,@InfoJakarta,Lansia dengan ktp daerah manapun bisa mendapat...,0,5,9,[],[],https://twitter.com/infojakarta/status/1372150...,False,,,,,[],[@infojakarta]
6,1371706352549040133,1371706352549040128,106038219,infojakarta,@InfoJakarta,"Pelaku industri pariwisata di Jakarta, apakah ...",0,0,3,[],[],https://twitter.com/infojakarta/status/1371706...,False,https://twitter.com/detikcom/status/1371701624...,,,,[],"[@infojakarta, @detikcom, Mar 16]"
7,1366579485793939456,1366579485793939456,106038219,infojakarta,@InfoJakarta,"Bergerak pararel. Vaksin jalan, pariwisata be...",0,1,2,[],[],https://twitter.com/infojakarta/status/1366579...,False,https://twitter.com/sandiuno/status/1366233283...,,,,[],"[@infojakarta, @sandiuno, Mar 1]"
8,1375866713879703555,1375866713879703552,106038219,infojakarta,@InfoJakarta,"Larangan Mudik 2021 ""Perihal operasional perj...",0,2,4,[],[],https://twitter.com/infojakarta/status/1375866...,False,,,,,[],[@infojakarta]
10,1372427544830435329,1372427544830435328,106038219,infojakarta,@InfoJakarta,"Mundurnya Timnas dari all england 2021. ""Anda...",3,0,4,[],[],https://twitter.com/infojakarta/status/1372427...,False,,,,,[],[@infojakarta]


In [8]:
arr = []
for i in data_tweets_after['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
arr.remove('@infojakarta')
arr.remove('@DKIJakarta')
arr.remove('@detikcom')
arr.remove('@sandiuno')
arr.remove('@kompascom')
arr.remove('@RadioElshinta')
print(len(arr))
arr

45


['@nitayaanita',
 '@oebunda',
 '@zakyBE',
 '@glowtweets',
 '@_banyulaut',
 '@sarahjoeri',
 '@liafrila',
 '@HerlambaRandy',
 '@ururleahciM',
 '@ap_perdana',
 '@dedienpratama',
 '@AnditoAndy',
 '@okkyjoeamstrong',
 '@Dyahwh',
 '@HasratMuliadi',
 '@gokamesa',
 '@eckaNh57',
 '@Lixxone',
 '@DonyYhudha',
 '@nofrizalwardoyo',
 '@melwth_',
 '@eriesatrya',
 '@Arie_16',
 '@bilonk',
 '@iamcakekhe',
 '@filterketengann',
 '@RemajaKolonial',
 '@dinkesJKT',
 '@riesbow',
 '@graciatbg',
 '@ruhiat_hilman',
 '@nesaa20',
 '@dont_sal',
 '@fyrdausy',
 '@herireh',
 '@masivian',
 '@agus_trimaulana',
 '@eriverde',
 '@siswandibambang',
 '@fettcinocreamyy',
 '@lutskay_',
 '@kokohbucin',
 '@wagesyahputra',
 '@chiko_jubilee',
 '@melanieppuchino']

In [9]:
filename = './data/after_user_data.json'
for uname in arr:
    uname = uname[1:]
    c = twint.Config()
    c.Username = uname
    c.Since = "2021-02-01" #batas awal tanggal pengambilan tweet
#     c.Until = "2021-01-31" #batas akhir tanggal pengambilan tweet
    c.Search = "covid OR vaksin"
    c.Count = True         
    c.Retweets = True
    c.Stats = True 
    c.Show_hashtags = True
    c.Hide_output = True
    c.Profile_full = True
    c.Store_json = True
    c.Output = filename # save to json
    twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 0 Tweets from @nitayaanita.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 0 Tweets from @oebunda.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 0 Tweets from @zakyBE.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 1 Tweets from @glowtweets.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 1 Tweets from @_banyulaut.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 0 Tweets from @sarahjoeri.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[+] Finished: Successfully collected 3 Tweets from @lia

In [10]:
after_user_data_tweets = pd.read_json('./data/after_user_data.json', lines = True)
after_user_data_tweets = after_user_data_tweets[['id', 'conversation_id', 'user_id', 'username', 'name','tweet','replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url','user_rt_id', 'user_rt', 'retweet_id', 'reply_to']]

after_user_data_tweets = after_user_data_tweets.drop_duplicates(subset='link')
for i in after_user_data_tweets['link']:
    print(i)

https://twitter.com/glowtweets/status/1373946472987717635
https://twitter.com/_banyulaut/status/1360530407414849536
https://twitter.com/liafrila/status/1381891367815708675
https://twitter.com/liafrila/status/1379070077010472963
https://twitter.com/liafrila/status/1373483235825516547
https://twitter.com/ururleahciM/status/1366676788764581890
https://twitter.com/ururleahciM/status/1364855526815768580
https://twitter.com/ap_perdana/status/1370215397069180930
https://twitter.com/ap_perdana/status/1358970719862857728
https://twitter.com/Dyahwh/status/1376939520340357127
https://twitter.com/HasratMuliadi/status/1361216322202296320
https://twitter.com/HasratMuliadi/status/1359859709608534016
https://twitter.com/DonyYhudha/status/1366587424017580038
https://twitter.com/nofrizalwardoyo/status/1376815767627296769
https://twitter.com/nofrizalwardoyo/status/1375756738528309249
https://twitter.com/nofrizalwardoyo/status/1372595164317851649
https://twitter.com/nofrizalwardoyo/status/1368960718897225

In [15]:
arr_temp = []
for i, row in after_user_data_tweets.iterrows():
     arr_temp.append(get_list_replies(row['link']))

after_user_data_tweets['list_replies'] = arr_temp
after_user_data_tweets['list_replies']

['@sicupuh', '@temponewsroom', 'Mar 21', '@glowtweets']
['@jokowi', '@_banyulaut']
['@bts_twt_thyung', '@liafrila']
['@BTS_twt__RM', '@liafrila', '@bts_twt_LN', '@liafrila']
['@BTS_twt_KTHV', '@liafrila']
['@ururleahciM']
['@drjeromekim1', '@ururleahciM']
['@garandra', '@ap_perdana']
['@JukiHoki', '@ap_perdana']
['@infojakarta', '@Dyahwh']
['@veghievenus', '@DKIJakarta', '@HasratMuliadi']
['@thedufresne', '@HasratMuliadi']
['@infojakarta', '@DonyYhudha']
['@nofrizalwardoyo', '@DKIJakarta', '@Riyan_bray']
['@kikysaputrii', '@nofrizalwardoyo', '@bharatahb', '@nofrizalwardoyo', '@potatoeggroll', '@KAI121', 'Mar 18', '@nofrizalwardoyo', '@gohzah']
['@nofrizalwardoyo']
['@nofrizalwardoyo', '@KemensetnegRI', 'Mar 8']
['@derarszt', '@nofrizalwardoyo', '@nindyasetya']
['@nofrizalwardoyo']
['@melwth_']
['@eriesatrya']
['@InfosuporterID', '@Arie_16']
['@kostumkomik', '@Arie_16']
['@Wotanet', '@Arie_16']
['@medcom_id', '@Arie_16']
['@KompasTV', '@Arie_16']
['@thisfrieska', '@thisfrieska', '@Arie_

0       [@sicupuh, @temponewsroom, Mar 21, @glowtweets]
1                                [@jokowi, @_banyulaut]
2                          [@bts_twt_thyung, @liafrila]
3     [@BTS_twt__RM, @liafrila, @bts_twt_LN, @liafrila]
4                            [@BTS_twt_KTHV, @liafrila]
                            ...                        
94                                   [@melanieppuchino]
95    [@tirta_hudhi, @siapadahnih, Mar 4, @melaniepp...
96                  [@melanieppuchino, @ANYudhp, Mar 2]
97                     [@FirzaHusain, @melanieppuchino]
98    [@melanieppuchino, @znovietha22, @melanieppuch...
Name: list_replies, Length: 99, dtype: object

In [16]:
after_user_data_tweets.to_json('./data/after_user_scrap.json')

In [17]:
after_user_data_tweets = pd.read_json('./data/after_user_scrap.json')
after_user_data_tweets.columns

Index(['id', 'conversation_id', 'user_id', 'username', 'name', 'tweet',
       'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'quote_url', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'list_replies'],
      dtype='object')

In [18]:
arr = []
for i in after_user_data_tweets['list_replies']:
    for x in i:
        arr.append(x)
arr = list(set(arr))

prefix = '@'
arr = [i for i in arr if prefix in i]
arr = list(set(arr))
print(len(arr))
arr

99


['@derarszt',
 '@bahasa_hanief',
 '@liafrila',
 '@afiiitz10',
 '@Eka4Liong',
 '@HasratMuliadi',
 '@ganjarpranowo',
 '@eriesatrya',
 '@mieke94447043',
 '@detikcom',
 '@bilonk',
 '@pringgaft',
 '@gohzah',
 '@rhmtk_',
 '@temponewsroom',
 '@dont_sal',
 '@herireh',
 '@SugiyantiNinik',
 '@tirta_hudhi',
 '@chiko_jubilee',
 '@ururleahciM',
 '@detikHealth',
 '@memekriot',
 '@potatoeggroll',
 '@KompasTV',
 '@Reisa_BA',
 '@kikysaputrii',
 '@daffa_samalam',
 '@FirzaHusain',
 '@ap_perdana',
 '@Dyananjani89',
 '@Leuncaaaa',
 '@BTS_twt_KTHV',
 '@iamcakekhe',
 '@infoBMKG',
 '@anaslyamardhia',
 '@znovietha22',
 '@thisfrieska',
 '@siswandibambang',
 '@kokohbucin',
 '@CNNIndonesia',
 '@veghievenus',
 '@iic_a',
 '@nyookkkk',
 '@JukiHoki',
 '@ANYudhp',
 '@glowtweets',
 '@Riyan_bray',
 '@_banyulaut',
 '@fransiscoarga1',
 '@makLambeTurah',
 '@medcom_id',
 '@drjeromekim1',
 '@Wotanet',
 '@richard_sasamu',
 '@jokowi',
 '@nofrizalwardoyo',
 '@Arie_16',
 '@garandra',
 '@tikaalmira',
 '@bts_twt_thyung',
 '@suarma